In [39]:
using DynamicsAndControl
using StaticArrays
using Rotations
using OrdinaryDiffEq
using Plots
plotlyjs()

┌ Info: Precompiling PlotlyJS [f0f68f2c-4968-5e81-91da-67840de0976a]
└ @ Base loading.jl:1278


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16912102848188411031\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8164774574750462595\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16912102848188411031\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

In [40]:
@dynamics PlanarRocketLanding{T} begin
    @integrable begin
        r::SVector{2, T} 
        v::SVector{2, T}
        θ::T
        ω::T
        m::T
    end

    @direct begin
        landed::Bool
    end
end

PlanarRocketLanding

In [41]:
@controller RocketLandingController{T} begin
    @outputs begin
        δ_gim::T
        T_thrust::T
    end
end

RocketLandingController

In [42]:
function DynamicsAndControl.initialize(::Type{PlanarRocketLanding}, config)
    x0 = config.x0

    return (x0.r, x0.v, x0.θ, x0.ω, x0.m), (false,), config.params
end

In [58]:
function DynamicsAndControl.dynamics!(this::PlanarRocketLanding, ẋ, x, u, t)
    @unpack Isp, J, t_hat_body, g0 = static(this)
    @unpack r, v, θ, ω, m, landed = x
    @unpack δ_gim, T_thrust = u

    f_thrust = T_thrust*RotMatrix(θ)*t_hat_body
    f_grav = @SVector([0.0, -m*g0])
    f_total = f_thrust + f_grav

    if !landed
        ẋ.r = v
        ẋ.v = f_total/m
        ẋ.θ = ω
        ẋ.ω = T_thrust*sin(δ_gim)/J
        ẋ.m = -T_thrust/(Isp*g0)
    else
        ẋ.r = ẋ.v = @SVector zeros(2)
        ẋ.θ = ẋ.ω = 0.0
    end

    log!(this, :state, t, (a=ẋ.v, ω̇=ẋ.ω, ṁ=ẋ.m, r, v, θ, ω, m))
    log!(this, :forces, t, (;f_thrust, f_grav, f_total))
end

In [44]:
DynamicsAndControl.initialize(::Type{RocketLandingController}, config) = (), (0.0, 0.0), config

In [66]:
function DynamicsAndControl.update!(this::RocketLandingController, u, _, x, t)
    @unpack m = x
    @unpack T_W, g0 = static(this)

    T_thrust = m*g0*T_W
    δ_gim = 0.0

    @pack! u = T_thrust, δ_gim

    log!(this, :outputs, t, (;T_thrust, δ_gim))
end

In [67]:
dynamics_conf = ( x0 = (
                         r = @SVector([.5e3, 2e3]), 
                         v = @SVector([-50.0, -250.0]),
                         θ = -20*π/180,
                         ω = .3*π/180,
                         m = 1e5,
                    ),
                  params = (
                        Isp = 300,
                        g0 = 9.81,
                        J = 1e6,
                        t_hat_body = @SVector [-1.0, 0.0]
                    )
                 )

(x0 = (r = [500.0, 2000.0], v = [-50.0, -250.0], θ = -0.3490658503988659, ω = 0.005235987755982988, m = 100000.0), params = (Isp = 300, g0 = 9.81, J = 1.0e6, t_hat_body = [-1.0, 0.0]))

In [68]:
control_conf = ( T_W = 1.1, g0 = 9.81)

(T_W = 1.1, g0 = 9.81)

In [69]:
sim = Simulation(
                    ( :truth, PlanarRocketLanding, dynamics_conf ),
                    ( :controller, RocketLandingController, control_conf ),
                    10.0, RK4(), dt=0.02
               )

Simulation{PlanarRocketLanding{DynamicsData{DynamicsAndControl.DynamicState{PlanarRocketLandingIntegrableState{Float64},PlanarRocketLandingIntegrableDerivative{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2},Float64,Float64,Float64},PlanarRocketLandingDirectState},Array{Float64,1},Tuple{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2},Float64,Float64,Float64},Tuple{Bool},ComponentData{Float64,NamedTuple{(:Isp, :g0, :J, :t_hat_body),Tuple{Int64,Float64,Float64,SArray{Tuple{2},Float64,1,2}}}}}},DynamicsAndControl.FullStateSensor{DynamicsAndControl.DynamicState{PlanarRocketLandingIntegrableState{Float64},PlanarRocketLandingIntegrableDerivative{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2},Float64,Float64,Float64},PlanarRocketLandingDirectState}},RocketLandingController{SensorActuatorControllerData{RocketLandingControllerState,RocketLandingControllerOutputs{Float64},Tuple{},ComponentData{Float64,NamedTuple{(:T_W, :g0),Tuple{Float64,Float64}}}}},DynamicsAndCon

In [70]:
data = simulate(sim)

Simulation data set with immediate namespaces:
	[:truth, :controller]


In [71]:
let p = data.truth.state
    #plot(p.time, p.r[1], xlabel="time", ylabel="horz")
    #plot!(p.time, p.r[2], xlabel="time", ylabel="vert")
    plot(p.r[1], p.r[2], xlabel="horz", ylabel="vert")
end

<!DOCTYPE html>
 
 
 Plots.jl

In [72]:
let p = data.truth.state
    plot(p.time, p.v)
end

<!DOCTYPE html>
 
 
 Plots.jl

In [72]:
let p = data.truth.state
    plot(p.time, p.v)
end

<!DOCTYPE html>
 
 
 Plots.jl

In [80]:
let p = data.truth.state
    plot(p.time, p.θ.*180/π)
end

<!DOCTYPE html>
 
 
 Plots.jl

In [77]:
let p = data.truth.state
    #plot(p.time, p.m, xlim=(0, .5), ylim=(0.0, 1.0))
    plot!(p.time, p.ṁ)
end

<!DOCTYPE html>
 
 
 Plots.jl